[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/neuralangelo-colab/blob/main/colmap_colab.ipynb)

In [ ]:
!apt-get install \
    cmake \
    libgoogle-glog-dev \
    libgflags-dev \
    libatlas-base-dev \
    libeigen3-dev \
    libsuitesparse-dev \
    libboost-program-options-dev \
    libboost-filesystem-dev \
    libboost-graph-dev \
    libboost-system-dev \
    libboost-test-dev \
    libfreeimage-dev \
    libmetis-dev \
    libglew-dev \
    qtbase5-dev \
    libqt5opengl5-dev \
    libcgal-dev

!wget https://huggingface.co/camenduru/neuralangelo/resolve/main/colmap.zip
!unzip /content/colmap.zip -d colmap

!cp -r /content/colmap/lib/. /usr/local/lib
!chmod 755 /content/colmap/bin/colmap
!cp -r /content/colmap/bin/. /usr/local/bin

!pip install -q commentjson addict
%cd /content

In [ ]:
# !wget https://huggingface.co/camenduru/neuralangelo/resolve/main/fox.zip -O /content/fox.zip
# !unzip /content/fox.zip -d /content

# !mkdir /content/fox/sparse /content/fox/dense

# !colmap feature_extractor \
#     --database_path /content/fox/database.db \
#     --image_path /content/fox \
#     --ImageReader.camera_model=RADIAL \
#     --SiftExtraction.use_gpu=true \
#     --SiftExtraction.num_threads=32 \
#     --ImageReader.single_camera=true # assuming single camera

# !colmap sequential_matcher \
#     --database_path /content/fox/database.db \
#     --SiftMatching.use_gpu=true

# !colmap mapper \
#     --database_path /content/fox/database.db \
#     --image_path /content/fox \
#     --output_path /content/fox/sparse

# !colmap image_undistorter \
#     --image_path /content/fox \
#     --input_path /content/fox/sparse/0 \
#     --output_path /content/fox/dense \
#     --output_type COLMAP \
#     --max_image_size 2000

!wget https://huggingface.co/camenduru/neuralangelo/resolve/main/lego.mp4 -O /content/lego.mp4
EXPERIMENT="lego"
PATH_TO_VIDEO="/content/lego.mp4"
SKIP_FRAME_RATE=2
SCENE_TYPE="object"
!bash /content/neuralangelo/projects/neuralangelo/scripts/preprocess.sh {EXPERIMENT} {PATH_TO_VIDEO} {SKIP_FRAME_RATE} {SCENE_TYPE}

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

# from google.colab import output
# output.disable_custom_widget_manager()

import numpy as np
import torch
import k3d
import json
from collections import OrderedDict
from projects.nerf.utils import camera, visualize
from third_party.colmap.scripts.python.read_write_model import read_model

colmap_path = "/content/neuralangelo/datasets/lego_skip2"
cameras, images, points_3D = read_model(path=f"{colmap_path}/dense/sparse", ext=".bin")
images = OrderedDict(sorted(images.items()))
qvecs = torch.from_numpy(np.stack([image.qvec for image in images.values()]))
tvecs = torch.from_numpy(np.stack([image.tvec for image in images.values()]))
Rs = camera.quaternion.q_to_R(qvecs)
poses = torch.cat([Rs, tvecs[..., None]], dim=-1)  # [N,3,4]
print(f"# images: {len(poses)}")
xyzs = torch.from_numpy(np.stack([point.xyz for point in points_3D.values()]))
rgbs = np.stack([point.rgb for point in points_3D.values()])
rgbs = (rgbs[:, 0] * 2**16 + rgbs[:, 1] * 2**8 + rgbs[:, 2]).astype(np.uint32)
print(f"# points: {len(xyzs)}")

json_fname = f"{colmap_path}/dense/transforms.json"
with open(json_fname) as file:
    meta = json.load(file)
center = meta["sphere_center"]
radius = meta["sphere_radius"]
readjust_center = np.array([0., 0., 0.])
readjust_scale = 0.25
center += readjust_center
radius *= readjust_scale
sphere_points = np.random.randn(100000, 3)
sphere_points = sphere_points / np.linalg.norm(sphere_points, axis=-1, keepdims=True)
sphere_points = sphere_points * radius + center

vis_scale = 0.5
plot = visualize.k3d_visualize_pose(poses,
                                    vis_depth=(0.5 * vis_scale),
                                    xyz_length=(0.1 * vis_scale),
                                    center_size=(0.05 * vis_scale),
                                    xyz_width=(0.02 * vis_scale))
plot += k3d.points(xyzs, colors=rgbs, point_size=(0.05 * vis_scale), shader="flat")
plot += k3d.points(sphere_points, color=0x4488ff, point_size=0.02, shader="flat")
plot.display()
plot.camera_fov = 30.0